In [3]:
#https://github.com/IDontEvenKnowCoding/ML-fundamentals-2025

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import zscore
import lightgbm as lgb


# Task 1: Exploratory Data Analysis (EDA)

In [ ]:
# Load the dataset and normalize column names to prevent hidden duplicates
data = pd.read_csv('hour.csv')

In [ ]:
data.columns = data.columns.str.strip().str.lower()  # Ensures consistent naming

In [ ]:
# Display the first few rows and info
print("Head of data:")
print(data.head())
print("\nData Info:")
print(data.info())
print("\nBasic Statistics:")
print(data.describe())

In [ ]:
# Plot target variable distribution (cnt)
plt.figure(figsize=(10, 6))
sns.histplot(data['cnt'], kde=True)
plt.title('Distribution of Bike Rentals Count')
plt.xlabel('Count')
plt.ylabel('Frequency')
plt.axvline(data['cnt'].mean(), color='red', linestyle='--', label=f"Mean: {data['cnt'].mean():.1f}")
plt.axvline(data['cnt'].median(), color='green', linestyle='--', label=f"Median: {data['cnt'].median()}")
plt.legend()
plt.show()

print(f"Skewness of target variable: {data['cnt'].skew()}")

In [ ]:
# Additional EDA plots (hour, weekday, month, season, weather, etc.)
plt.figure(figsize=(12, 6))
sns.boxplot(x='hr', y='cnt', data=data)
plt.title('Bike Rentals by Hour')
plt.xlabel('Hour of the Day')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 6))
sns.boxplot(x='weekday', y='cnt', data=data)
plt.title('Bike Rentals by Day of Week')
plt.xlabel('Day of Week')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 6))
sns.boxplot(x='mnth', y='cnt', data=data)
plt.title('Bike Rentals by Month')
plt.xlabel('Month')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 6))
sns.boxplot(x='season', y='cnt', data=data)
plt.title('Bike Rentals by Season')
plt.xlabel('Season (1:Winter, 2:Spring, 3:Summer, 4:Fall)')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

In [ ]:
# Weather-related scatter plots
weather_features = ['temp', 'atemp', 'hum', 'windspeed']
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()
for i, feature in enumerate(weather_features):
    sns.scatterplot(x=feature, y='cnt', data=data, alpha=0.3, ax=axes[i])
    axes[i].set_title(f'Bike Rentals vs {feature}')
    axes[i].set_xlabel(feature)
    axes[i].set_ylabel('Count')
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 6))
sns.boxplot(x='weathersit', y='cnt', data=data)
plt.title('Bike Rentals by Weather Situation')
plt.xlabel('Weather Situation (1:Clear, 2:Mist, 3:Light Snow/Rain)')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

# Convert dteday to datetime for correlation analysis
data['dteday'] = pd.to_datetime(data['dteday'])

# Correlation analysis
plt.figure(figsize=(14, 10))
correlation_matrix = data.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation Matrix')
plt.tight_layout()
plt.show()

# Check correlation between temp and atemp specifically
print(f"Correlation between temp and atemp: {correlation_matrix.loc['temp', 'atemp']:.4f}")

In [ ]:
# Calculate IQR-based bounds for 'temp'
Q1 = data['temp'].quantile(0.25)
Q3 = data['temp'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Plot the distribution of 'temp' with outlier bounds
plt.figure(figsize=(10, 6))
sns.histplot(data['temp'], bins=30, kde=True)
plt.axvline(lower_bound, color='red', linestyle='--', label=f'Lower Bound: {lower_bound:.2f}')
plt.axvline(upper_bound, color='green', linestyle='--', label=f'Upper Bound: {upper_bound:.2f}')
plt.title('Temperature Distribution with IQR Outlier Bounds')
plt.xlabel('Temperature')
plt.legend()
plt.show()

# Calculate IQR for 'windspeed'
Q1_windspeed = data['windspeed'].quantile(0.25)
Q3_windspeed = data['windspeed'].quantile(0.75)
IQR_windspeed = Q3_windspeed - Q1_windspeed
lower_bound_windspeed = Q1_windspeed - 1.5 * IQR_windspeed
upper_bound_windspeed = Q3_windspeed + 1.5 * IQR_windspeed
# Plot the distribution of 'windspeed' with outlier bounds
plt.figure(figsize=(10, 6))
sns.histplot(data['windspeed'], bins=30, kde=True)
plt.axvline(lower_bound_windspeed, color='red', linestyle='--', label=f'Lower Bound: {lower_bound_windspeed:.2f}')
plt.axvline(upper_bound_windspeed, color='green', linestyle='--', label=f'Upper Bound: {upper_bound_windspeed:.2f}')
plt.title('Windspeed Distribution with IQR Outlier Bounds')
plt.xlabel('Windspeed')
plt.legend()
plt.show() 

#Calculate IQR for 'hum'
Q1_hum = data['hum'].quantile(0.25)
Q3_hum = data['hum'].quantile(0.75)
IQR_hum = Q3_hum - Q1_hum
lower_bound_hum = Q1_hum - 1.5 * IQR_hum
upper_bound_hum = Q3_hum + 1.5 * IQR_hum
# Plot the distribution of 'hum' with outlier bounds
plt.figure(figsize=(10, 6))
sns.histplot(data['hum'], bins=30, kde=True)
plt.axvline(lower_bound_hum, color='red', linestyle='--', label=f'Lower Bound: {lower_bound_hum:.2f}')
plt.axvline(upper_bound_hum, color='green', linestyle='--', label=f'Upper Bound: {upper_bound_hum:.2f}')
plt.title('Humidity Distribution with IQR Outlier Bounds')
plt.xlabel('Humidity')
plt.legend()
plt.show()

# Outlier detection using z-score
from scipy.stats import zscore

# Compute the z-score for 'cnt'
data['cnt_zscore'] = zscore(data['cnt'])

# Plot a scatter plot of bike rentals over time with outliers highlighted (z-score > 3)
plt.figure(figsize=(10, 6))
sns.scatterplot(x=data.index, y='cnt', data=data, hue=data['cnt_zscore'] > 3, palette={True: 'red', False: 'blue'})
plt.title('Bike Rentals Over Time with Outliers Highlighted (z-score > 3)')
plt.xlabel('Record Index')
plt.ylabel('Bike Rental Count')
plt.show()

# Task 2: Data Splitting

In [ ]:
# Remove any temporary columns (like z-score for cnt) if they exist
if 'cnt_zscore' in data.columns:
    data = data.drop(columns=['cnt_zscore'])

# Convert dteday to numeric ordinal and sort the data
data['dteday'] = data['dteday'].map(pd.Timestamp.toordinal)
data = data.sort_values(by='dteday')

# Sequential split into train (60%), validation (20%), and test (20%)
n = len(data)
train = data.iloc[:int(0.6 * n)].copy()
val   = data.iloc[int(0.6 * n):int(0.8 * n)].copy()
test  = data.iloc[int(0.8 * n):].copy()

print("Training set size:", train.shape[0])
print("Validation set size:", val.shape[0])
print("Test set size:", test.shape[0])

In [ ]:
# Keep full copies for later use
X_train_full = train.copy()
X_val_full   = val.copy()
X_test_full  = test.copy()

# Define target variable y
y_train = train['cnt']
y_val   = val['cnt']
y_test  = test['cnt']

# Check for missing seasons in training and adjust if necessary
train_seasons = set(X_train_full['season'].unique())
all_seasons = set(data['season'].unique())
missing_seasons = all_seasons - train_seasons
if missing_seasons:
    print("Missing seasons in training data:", missing_seasons)
    rows_to_move = X_val_full[X_val_full['season'].isin(missing_seasons)]
    X_train_full = pd.concat([X_train_full, rows_to_move])
    y_train = pd.concat([y_train, y_val.loc[rows_to_move.index]])
    X_val_full = X_val_full.drop(rows_to_move.index)
    y_val = y_val.drop(rows_to_move.index)
    print("Updated training seasons:", set(X_train_full['season'].unique()))
else:
    print("All seasons are already present in the training data.")

# Drop unnecessary columns (target and others not needed for modeling)
columns_to_drop = ['instant', 'dteday', 'casual', 'registered', 'atemp', 'cnt']
X_train = X_train_full.drop(columns=columns_to_drop)
X_val   = X_val_full.drop(columns=columns_to_drop)
X_test  = X_test_full.drop(columns=columns_to_drop)
print("Final training feature columns:", X_train.columns.tolist())

# Task 3: Feature Engineering

In [ ]:
# Define feature groups
numerical_features = ['temp', 'hum', 'windspeed']
categorical_features = ['season', 'mnth', 'weathersit']
cyclical_features = ['hr', 'weekday']

# Function to encode cyclical features
def encode_cyclical_features(df, col, max_val):
    df[f'{col}_sin'] = np.sin(2 * np.pi * df[col] / max_val)
    df[f'{col}_cos'] = np.cos(2 * np.pi * df[col] / max_val)
    return df


In [ ]:
# Create copies for encoding
X_train_encoded = X_train.copy()
X_val_encoded   = X_val.copy()
X_test_encoded  = X_test.copy()

# Encode cyclical features for 'hr' and 'weekday'
for col, max_val in [('hr', 24), ('weekday', 7)]:
    X_train_encoded = encode_cyclical_features(X_train_encoded, col, max_val)
    X_val_encoded   = encode_cyclical_features(X_val_encoded, col, max_val)
    X_test_encoded  = encode_cyclical_features(X_test_encoded, col, max_val)

# One-hot encode categorical features – reinitialize encoder per column
for cat_feature in categorical_features:
    encoder = OneHotEncoder(sparse_output=False, drop='first')
    # Fit and transform on training data
    encoded_train = encoder.fit_transform(X_train_encoded[[cat_feature]])
    feature_names = [f"{cat_feature}_{i}" for i in range(encoded_train.shape[1])]
    encoded_df_train = pd.DataFrame(encoded_train, columns=feature_names, index=X_train_encoded.index)
    X_train_encoded = pd.concat([X_train_encoded, encoded_df_train], axis=1)

In [ ]:
# Transform validation and test data
encoded_val = encoder.transform(X_val_encoded[[cat_feature]])
encoded_df_val = pd.DataFrame(encoded_val, columns=feature_names, index=X_val_encoded.index)
X_val_encoded = pd.concat([X_val_encoded, encoded_df_val], axis=1)
    
encoded_test = encoder.transform(X_test_encoded[[cat_feature]])
encoded_df_test = pd.DataFrame(encoded_test, columns=feature_names, index=X_test_encoded.index)
X_test_encoded = pd.concat([X_test_encoded, encoded_df_test], axis=1)

# Felt that my results for models were too good to be true, so I added a leakage chec
# ======= LEAKAGE / Consistency Check for One-Hot Encoding =======
for cat in categorical_features:
    encoder = OneHotEncoder(sparse_output=False, drop='first')
    train_encoded = encoder.fit_transform(X_train[[cat]])
    val_encoded = encoder.transform(X_val[[cat]])
    feature_names = [f"{cat}_{i}" for i in range(train_encoded.shape[1])]
    train_df = pd.DataFrame(train_encoded, columns=feature_names, index=X_train.index)
    val_df = pd.DataFrame(val_encoded, columns=feature_names, index=X_val.index)
    # This assertion ensures that the same dummy columns are created for both sets
    assert list(train_df.columns) == list(val_df.columns), f"Mismatch in encoding for {cat}"
    print(f"Encoding for '{cat}' is consistent. Columns: {list(train_df.columns)}")
# ======= End Leakage Check =======

# Drop original categorical and cyclical columns
cols_to_drop = categorical_features + cyclical_features
X_train_encoded = X_train_encoded.drop(columns=cols_to_drop)
X_val_encoded   = X_val_encoded.drop(columns=cols_to_drop)
X_test_encoded  = X_test_encoded.drop(columns=cols_to_drop)

# Scale numerical features
scaler = StandardScaler()
X_train_encoded[numerical_features] = scaler.fit_transform(X_train_encoded[numerical_features])
X_val_encoded[numerical_features]   = scaler.transform(X_val_encoded[numerical_features])
X_test_encoded[numerical_features]    = scaler.transform(X_test_encoded[numerical_features])

In [ ]:
print(set(X_train.index) & set(X_val.index))  # should be empty


In [ ]:
# Create interaction term: temp * hum
interaction_name = 'temp_hum'

# If duplicate exists, rename the new interaction column
if interaction_name in X_train_encoded.columns: interaction_name = 'temp_hum_int'
X_train_encoded[interaction_name] = X_train_encoded['temp'] * X_train_encoded['hum']
X_val_encoded[interaction_name]   = X_val_encoded['temp'] * X_val_encoded['hum']
X_test_encoded[interaction_name]  = X_test_encoded['temp'] * X_test_encoded['hum']

print("Feature engineering completed.")
print(f"Training features shape: {X_train_encoded.shape}")
print(f"Feature names: {X_train_encoded.columns.tolist()}")


In [ ]:
# Make validation and test sets match the training set
X_val_encoded = X_val_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)
X_test_encoded = X_test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)


# Task 4: Baseline Model – Linear Regression

In [ ]:
lr_model = LinearRegression()
lr_model.fit(X_train_encoded, y_train)
y_train_pred_lr = lr_model.predict(X_train_encoded)
y_val_pred_lr = lr_model.predict(X_val_encoded)

# Ensure the validation set has the same columns as the training set
X_val_encoded = X_val_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)
y_val_pred_lr = lr_model.predict(X_val_encoded)

# Compute residuals from the baseline model predictions
residuals_lr = y_val_pred_lr - y_val

def evaluate_model(y_true, y_pred, name="Model"):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"{name} Performance:")
    print(f"  MSE: {mse:.2f}")
    print(f"  RMSE: {rmse:.2f}")
    print(f"  MAE: {mae:.2f}")
    print(f"  R² Score: {r2:.4f}\n")
    return mse, rmse, mae, r2

print("Linear Regression Training Results:")
lr_train_metrics = evaluate_model(y_train, y_train_pred_lr, "Linear Regression (Training)")
print("Linear Regression Validation Results:")
lr_val_metrics = evaluate_model(y_val, y_val_pred_lr, "Linear Regression (Validation)")



# Task 5: Random Forest Regressor

In [ ]:
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train_encoded, y_train)
y_train_pred_rf = rf_model.predict(X_train_encoded)
y_val_pred_rf = rf_model.predict(X_val_encoded)

print("Random Forest Training Results:")
rf_train_metrics = evaluate_model(y_train, y_train_pred_rf, "Random Forest (Training)")
print("Random Forest Validation Results:")
rf_val_metrics = evaluate_model(y_val, y_val_pred_rf, "Random Forest (Validation)")

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.scatter(y_val_pred_rf, y_val_pred_rf - y_val, alpha=0.5)
plt.axhline(0, color='red', linestyle='-')
plt.title('RF Residuals vs Predicted')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.subplot(1, 2, 2)
sns.histplot(y_val_pred_rf - y_val, kde=True)
plt.title('RF Residual Distribution')
plt.xlabel('Residuals')
plt.tight_layout()
plt.show()

rf_feature_importance = pd.DataFrame({
    'Feature': X_train_encoded.columns,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)
plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=rf_feature_importance.head(15))
plt.title('RF Feature Importance (Top 15)')
plt.tight_layout()
plt.show()


# Task 6: Gradient Boosting Regressor (LightGBM)

In [ ]:
gb_model = GradientBoostingRegressor(random_state=42)
gb_model.fit(X_train_encoded, y_train)
y_train_pred_gb = gb_model.predict(X_train_encoded)
y_val_pred_gb = gb_model.predict(X_val_encoded)

print("Gradient Boosting Training Results:")
gb_train_metrics = evaluate_model(y_train, y_train_pred_gb, "Gradient Boosting (Training)")
print("Gradient Boosting Validation Results:")
gb_val_metrics = evaluate_model(y_val, y_val_pred_gb, "Gradient Boosting (Validation)")

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.scatter(y_val_pred_gb, y_val_pred_gb - y_val, alpha=0.5)
plt.axhline(0, color='red', linestyle='-')
plt.title('GB Residuals vs Predicted')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.subplot(1, 2, 2)
sns.histplot(y_val_pred_gb - y_val, kde=True)
plt.title('GB Residual Distribution')
plt.xlabel('Residuals')
plt.tight_layout()
plt.show()

gb_feature_importance = pd.DataFrame({
    'Feature': X_train_encoded.columns,
    'Importance': gb_model.feature_importances_
}).sort_values(by='Importance', ascending=False)
plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=gb_feature_importance.head(15))
plt.title('GB Feature Importance (Top 15)')
plt.tight_layout()
plt.show()

In [ ]:
# LightGBM model with default parameters
lgb_model = lgb.LGBMRegressor(random_state=42)
lgb_model.fit(X_train_encoded, y_train)
y_train_pred_lgb = lgb_model.predict(X_train_encoded)
y_val_pred_lgb = lgb_model.predict(X_val_encoded)

print("LightGBM Training Results:")
lgb_train_metrics = evaluate_model(y_train, y_train_pred_lgb, "LightGBM (Training)")
print("LightGBM Validation Results:")
lgb_val_metrics = evaluate_model(y_val, y_val_pred_lgb, "LightGBM (Validation)")

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.scatter(y_val_pred_lgb, y_val_pred_lgb - y_val, alpha=0.5)
plt.axhline(0, color='red', linestyle='-')
plt.title('LightGBM Residuals vs Predicted')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.subplot(1, 2, 2)
sns.histplot(y_val_pred_lgb - y_val, kde=True)
plt.title('LightGBM Residual Distribution')
plt.xlabel('Residuals')
plt.tight_layout()
plt.show()

lgb_feature_importance = pd.DataFrame({
    'Feature': X_train_encoded.columns,
    'Importance': lgb_model.feature_importances_
}).sort_values(by='Importance', ascending=False)
plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=lgb_feature_importance.head(15))
plt.title('LightGBM Feature Importance (Top 15)')
plt.tight_layout()
plt.show()



Here we see that lightgbm is better than normal scikit-learns gradientboosting model.


# Task 7: Hyperparameter Tuning

In [ ]:
# Tuning Random Forest Regressor
rf_param_grid = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_random = RandomizedSearchCV(
    estimator = RandomForestRegressor(random_state=42),
    param_distributions = rf_param_grid,
    n_iter = 20,
    cv = 5,
    verbose = 1,
    random_state = 42,
    n_jobs = -1,
    scoring = 'neg_mean_squared_error'
)

rf_random.fit(X_train_encoded, y_train)
print("Best Random Forest Parameters:")
print(rf_random.best_params_)

best_rf_model = rf_random.best_estimator_
y_val_pred_rf_tuned = best_rf_model.predict(X_val_encoded)
print("Tuned Random Forest Validation Results:")
rf_tuned_val_metrics = evaluate_model(y_val, y_val_pred_rf_tuned, "Tuned Random Forest (Validation)")

rf_tuned_feature_importance = pd.DataFrame({
    'Feature': X_train_encoded.columns,
    'Importance': best_rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)
plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=rf_tuned_feature_importance.head(15))
plt.title('Tuned Random Forest Feature Importance (Top 15)')
plt.tight_layout()
plt.show()

In [ ]:
# Tuning LightGBM using RandomizedSearchCV
lgb_param_grid = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [3, 4, 5, 6, 7],
    'num_leaves': [15, 31, 63],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0]
}

lgb_random = RandomizedSearchCV(
    estimator=lgb.LGBMRegressor(random_state=42),
    param_distributions=lgb_param_grid,
    n_iter=20,
    cv=5,
    verbose=1,
    random_state=42,
    n_jobs=-1,
    scoring='neg_mean_squared_error'
)

lgb_random.fit(X_train_encoded, y_train)
print("Best LightGBM Parameters:")
print(lgb_random.best_params_)

# Retrieve the best estimator using the tuned hyperparameters
best_lgb_model = lgb_random.best_estimator_
y_val_pred_lgb_tuned = best_lgb_model.predict(X_val_encoded)

print("Tuned LightGBM Validation Results:")
lgb_tuned_val_metrics = evaluate_model(y_val, y_val_pred_lgb_tuned, "Tuned LightGBM (Validation)")

# Plot feature importance from the tuned LightGBM model
lgb_tuned_feature_importance = pd.DataFrame({
    'Feature': X_train_encoded.columns,
    'Importance': best_lgb_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=lgb_tuned_feature_importance.head(15))
plt.title('Tuned LightGBM Feature Importance (Top 15)')
plt.tight_layout()
plt.show()

In [ ]:
# Comparison of Models on Validation Set
comparison = pd.DataFrame({
    'Model': ['Linear Regression', 'RF (Default)', 'RF (Tuned)', 'LGBM (Default)', 'LGBM (Tuned)'],
    'MSE': [lr_val_metrics[0], rf_val_metrics[0], rf_tuned_val_metrics[0], lgb_val_metrics[0], lgb_tuned_val_metrics[0]],
    'RMSE': [lr_val_metrics[1], rf_val_metrics[1], rf_tuned_val_metrics[1], lgb_val_metrics[1], lgb_tuned_val_metrics[1]],
    'MAE': [lr_val_metrics[2], rf_val_metrics[2], rf_tuned_val_metrics[2], lgb_val_metrics[2], lgb_tuned_val_metrics[2]],
    'R²': [lr_val_metrics[3], rf_val_metrics[3], rf_tuned_val_metrics[3], lgb_val_metrics[3], lgb_tuned_val_metrics[3]]
})
print("Comparison of Models on Validation Set:")
print(comparison)

# Task 8: Iterative Evaluation and Refinement


In [ ]:
#Refining the Linear Regression model by addressing outliers in the training data

# Identify outliers in the validation residuals using the z-score method
residuals_z = np.abs(zscore(residuals_lr))
outlier_val_indices = np.where(residuals_z > 3)[0]
print("Number of outliers in validation residuals:", len(outlier_val_indices))

# 1. Evaluate training data for potential outliers in the target variable
# If a significant number of outliers are detected in training, consider removing them.
y_train_z = np.abs(zscore(y_train))
outlier_train_indices = np.where(y_train_z > 3)[0]
print("Number of outliers in training target:", len(outlier_train_indices))

# Here, we create refined training sets by removing identified outliers.
X_train_refined = X_train_encoded.copy()
y_train_refined = y_train.copy()
if len(outlier_train_indices) > 0:
    X_train_refined = X_train_refined.drop(index=X_train_refined.index[outlier_train_indices])
    y_train_refined = y_train_refined.drop(index=y_train_refined.index[outlier_train_indices])
    print("Refined training set size:", X_train_refined.shape[0])
else:
    print("No significant outliers detected in training set.")

# 2. Retrain the Linear Regression model on the refined training set
lr_model_refined = LinearRegression()
lr_model_refined.fit(X_train_refined, y_train_refined)
y_val_pred_lr_refined = lr_model_refined.predict(X_val_encoded)

# 3. Evaluate the refined model on the original validation set
print("Linear Regression (Refined) Validation Results:")
lr_refined_metrics = evaluate_model(y_val, y_val_pred_lr_refined, "Linear Regression (Refined)")

# 4. Analyze the validation residuals from the initial Linear Regression model
residuals_lr = y_val_pred_lr - y_val
plt.figure(figsize=(8, 6))
sns.histplot(residuals_lr, kde=True)
plt.title("Linear Regression Residual Distribution (Before Refinement)")
plt.xlabel("Residuals")
plt.show()

# 5. Plot the residuals of the refined model for comparison
plt.figure(figsize=(8, 6))
sns.histplot(y_val_pred_lr_refined - y_val, kde=True)
plt.title("Linear Regression Residual Distribution (After Refinement)")
plt.xlabel("Residuals")
plt.show()

In [ ]:
comparison_lr = pd.DataFrame({
    'Metric': ["MSE", "RMSE", "MAE", "R² Score"],
    'Before Refinement': lr_val_metrics,
    'After Refinement': lr_refined_metrics
})
print(comparison_lr)

Doesn't seem to provide better performance

In [ ]:
# (a) Evaluate the original Random Forest model on training data and identify outliers
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train_encoded, y_train)
y_train_pred_rf = rf_model.predict(X_train_encoded)

# Identify outliers in the training target using z-score
rf_train_z = np.abs(zscore(y_train))
outlier_rf_indices = np.where(rf_train_z > 3)[0]
print("Random Forest: Number of outliers in training:", len(outlier_rf_indices))

# Create refined training set by removing outlier indices
X_train_rf_refined = X_train_encoded.copy()
y_train_rf_refined = y_train.copy()
if len(outlier_rf_indices) > 0:
    X_train_rf_refined = X_train_rf_refined.drop(index=X_train_rf_refined.index[outlier_rf_indices])
    y_train_rf_refined = y_train_rf_refined.drop(index=y_train_rf_refined.index[outlier_rf_indices])
    print("Refined training size for Random Forest:", X_train_rf_refined.shape[0])
else:
    print("No outliers detected for Random Forest refinement.")

# Retrain Random Forest on the refined training set and evaluate on validation
rf_model_refined = RandomForestRegressor(random_state=42)
rf_model_refined.fit(X_train_rf_refined, y_train_rf_refined)
y_val_pred_rf_refined = rf_model_refined.predict(X_val_encoded)
print("\nRandom Forest (Refined) Validation Results:")
rf_refined_metrics = evaluate_model(y_val, y_val_pred_rf_refined, "Random Forest (Refined)")

In [ ]:
# (a) Evaluate the original LightGBM model on training data and identify outliers
lgb_model = lgb.LGBMRegressor(random_state=42)
lgb_model.fit(X_train_encoded, y_train)
y_train_pred_lgb = lgb_model.predict(X_train_encoded)

# Identify outliers in the training target using z-score
lgb_train_z = np.abs(zscore(y_train))
outlier_lgb_indices = np.where(lgb_train_z > 3)[0]
print("LightGBM: Number of outliers in training:", len(outlier_lgb_indices))

# Create refined training set by removing outlier indices
X_train_lgb_refined = X_train_encoded.copy()
y_train_lgb_refined = y_train.copy()
if len(outlier_lgb_indices) > 0:
    X_train_lgb_refined = X_train_lgb_refined.drop(index=X_train_lgb_refined.index[outlier_lgb_indices])
    y_train_lgb_refined = y_train_lgb_refined.drop(index=y_train_lgb_refined.index[outlier_lgb_indices])
    print("Refined training size for LightGBM:", X_train_lgb_refined.shape[0])
else:
    print("No outliers detected for LightGBM refinement.")

# Retrain LightGBM on the refined training set and evaluate on validation
lgb_model_refined = lgb.LGBMRegressor(random_state=42)
lgb_model_refined.fit(X_train_lgb_refined, y_train_lgb_refined)
y_val_pred_lgb_refined = lgb_model_refined.predict(X_val_encoded)
print("\nLightGBM (Refined) Validation Results:")
lgb_refined_metrics = evaluate_model(y_val, y_val_pred_lgb_refined, "LightGBM (Refined)")


In [ ]:
# For demonstration, we use dummy lists; update them accordingly.
rf_metrics_before = [rf_train_metrics[0], rf_train_metrics[1], rf_train_metrics[2], rf_train_metrics[3]]  # e.g., [MSE, RMSE, MAE, R²]
rf_metrics_after  = [rf_refined_metrics[0], rf_refined_metrics[1], rf_refined_metrics[2], rf_refined_metrics[3]]

lgb_metrics_before = [lgb_train_metrics[0], lgb_train_metrics[1], lgb_train_metrics[2], lgb_train_metrics[3]]
lgb_metrics_after  = [lgb_refined_metrics[0], lgb_refined_metrics[1], lgb_refined_metrics[2], lgb_refined_metrics[3]]

comparison_rf = pd.DataFrame({
    "Metric": ["MSE", "RMSE", "MAE", "R² Score"],
    "Random Forest Before": rf_metrics_before,
    "Random Forest After": rf_metrics_after
})
comparison_lgb = pd.DataFrame({
    "Metric": ["MSE", "RMSE", "MAE", "R² Score"],
    "LightGBM Before": lgb_metrics_before,
    "LightGBM After": lgb_metrics_after
})

print("\nRandom Forest Comparison (Before vs. After Refinement):")
print(comparison_rf)
print("\nLightGBM Comparison (Before vs. After Refinement):")
print(comparison_lgb)

Seems like over fitting for the training data? I did a check to ensure no leakage in step 3 but still getting the exact same values before. Very large difference 

# Task 9: Final Model Selection and Testing

In [ ]:
# Compute validation metrics for each model if not already defined
lr_metrics_original = evaluate_model(y_val, y_val_pred_lr, "Linear Regression (Original)")
rf_metrics_original = evaluate_model(y_val, y_val_pred_rf, "Random Forest (Original)")
lgb_metrics_original = evaluate_model(y_val, y_val_pred_lgb, "LightGBM (Original)")





REALLY LOW error rates for random forest & LightGBM model, asked gpt but it said it could be due to over fitting instead. Still trying to find if any/where data leakage could have occured.

In [ ]:
# Combine training and validation sets
X_train_val = pd.concat([X_train_encoded, X_val_encoded], axis=0)
y_train_val = pd.concat([y_train, y_val], axis=0)

# Retrain best model (LightGBM) on full train+val set
final_lgb_model = lgb.LGBMRegressor(random_state=42)
final_lgb_model.fit(X_train_val, y_train_val)

In [ ]:
# Check which columns are missing in validation
missing_in_val = set(X_train_encoded.columns) - set(X_val_encoded.columns)
extra_in_val = set(X_val_encoded.columns) - set(X_train_encoded.columns)

print(" Missing in validation (present in training):", missing_in_val)
print(" Extra in validation (not in training):", extra_in_val)


In [ ]:
# Combine training + validation
X_train_val = pd.concat([X_train_encoded, X_val_encoded])
y_train_val = pd.concat([y_train, y_val])

# Align test set again (in case train+val added new columns)
X_test_encoded = X_test_encoded.reindex(columns=X_train_val.columns, fill_value=0)

In [ ]:
# Evaluate on test set
y_test_pred = final_lgb_model.predict(X_test_encoded)

final_mse = mean_squared_error(y_test, y_test_pred)
final_rmse = np.sqrt(final_mse)
final_mae = mean_absolute_error(y_test, y_test_pred)
final_r2 = r2_score(y_test, y_test_pred)

print(" Final Model Evaluation on Test Set:")
print(f"  MSE: {final_mse:.2f}")
print(f"  RMSE: {final_rmse:.2f}")
print(f"  MAE: {final_mae:.2f}")
print(f"  R² Score: {final_r2:.4f}")

# Plot residuals
plt.figure(figsize=(12, 5))

# Residuals vs Predicted
plt.subplot(1, 2, 1)
plt.scatter(y_test_pred, y_test_pred - y_test, alpha=0.5)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuals vs Predicted (Test Set)')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')

# Distribution of Residuals
plt.subplot(1, 2, 2)
sns.histplot(y_test_pred - y_test, kde=True)
plt.title('Residual Distribution (Test Set)')
plt.xlabel('Residuals')

plt.tight_layout()
plt.show()

The model generalises well compared to our baseline models. Tunning by RandomizedSearchCV has improved generalization by optimizing critical hyperparameters.

Never trained a model like this, but it feels like the model is doing really well. Is this possible?